In [1]:
%%time
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import wave
import numpy
from music21 import converter, instrument, note, chord
import glob
import os
import Paths
from pydub import AudioSegment
from keras.models import Sequential
from keras.layers import Dense, LSTM, MaxPooling2D, Dropout, Activation
import librosa
import tensorflow as tf
from keras.layers import LeakyReLU

directory = os.listdir(Paths.IN_DIRECTORY)
array = []

sequence_length = []
for sample_name in directory:
    if 'wav' in sample_name:
        file_path = os.path.join(Paths.IN_DIRECTORY, sample_name)
        song, sr = librosa.load(file_path)
        sequence_length.append(len(song))
        #print(len(song))
        #song = AudioSegment.from_wav(file_path)
        #samples = song.get_array_of_samples()
        array.append(song)

# Removing the last element using slicing
lastElementIndex = len(array)-1
in_array = array[:lastElementIndex]
out_array = array[1:]


# Create the Input and Output to tensors
input_tensor = tf.convert_to_tensor(in_array)
output_tensor = tf.convert_to_tensor(out_array)

batch_size = len(input_tensor)
timestep = sequence_length[0]

input_data = tf.reshape(input_tensor, shape=(batch_size, timestep, 1))
output_data = tf.reshape(output_tensor, shape=(batch_size, timestep, 1))

# one hot encode the output variable
#y = np_utils.to_categorical(output_data)

#Normalize the training data based on overall number of values
#input_data = input_data / Number_of_values

model = Sequential()
first_lstm = LSTM(32,
        batch_input_shape=(batch_size, timestep, 1),
        return_sequences=True, activation='tanh',
        stateful=True)

model.add(first_lstm)
model.add(LeakyReLU())
model.add(Dropout(0.2))

#this is the last LSTM layer, use return_sequences=False
model.add(LSTM(16, return_sequences=False, stateful=True,  activation='tanh'))
model.add(Dropout(0.2))
model.add(LeakyReLU())
model.add(Dense(1))
model.compile(loss='binary_crossentropy', optimizer='rmsprop')
model.fit(input_data, output_data, epochs=2, batch_size=sequence_length[0], verbose=2)





Epoch 1/2
1/1 - 166s - loss: -1.9261e-02 - 166s/epoch - 166s/step
Epoch 2/2
1/1 - 162s - loss: -1.9286e-02 - 162s/epoch - 162s/step
CPU times: total: 6min 59s
Wall time: 5min 47s


In [12]:
# demonstrate prediction

x_input = array[0]
print(x_input.shape)
#x_input = numpy.array(list(map(numpy.float, x_input)))
x_input = x_input.reshape((1, timestep, 1))
print(x_input.shape)
yhat = model.predict(x_input, batch_size=sequence_length[0], verbose=2)
print(yhat)

(66150,)
(1, 66150, 1)


InvalidArgumentError: Graph execution error:

Specified a list with shape [9,1] from a tensor with shape [1,1]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_predict_function_5701]

In [ ]:

model = Sequential()
model.add(LSTM(units= 2, batch_input_shape= [9, 66150, 1], stateful=True, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(32, return_sequences=True))
#model.add(Dropout(0.3))
#model.add(LSTM(32))
#model.add(Dense(32))
#model.add(Dropout(0.3))
#model.add(Dense(n_vocab))
#model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

model.fit(input_data, input_data, epochs=200, batch_size=sequence_length[0], callbacks=callbacks_list)


In [ ]:
'''

 sequence_in = flatlist[i:i + sequence_length[i]]
    sequence_out = flatlist[i + sequence_length[i]]
    network_input.append(sequence_in)
    network_output.append(sequence_out)
n_patterns = len(network_input)

# sequence_length = len(song)
sequence_length = 100

# get all pitch names
pitchnames = sorted(set(item for item in flatlist))

n_vocab = len(set(flatlist))

# create a dictionary to map pitches to integers
note_to_int = dict((flatlist, number) for number, flatlist in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(flatlist) - sequence_length, 1):
    sequence_in = flatlist[i:i + sequence_length]
    sequence_out = flatlist[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)


# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

'''

In [ ]:
#print(network_input)
print(array)


In [ ]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('weights-improvement-59-2.5657-bigger.hdf5')


In [ ]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
import torchaudio
import torch
import numpy


torchaudio.set_audio_backend('sox')  # sox is the default

waveform, sample_rate = torchaudio.load('C:/Users/hennm/Dropbox/Studium/Master/Semester 2/Tec Lab/Music/Samples/Tea K Pea - nauticals_1.wav')

downsample_rate=8000

downsample_resample = torchaudio.transforms.Resample(
    sample_rate, downsample_rate, resampling_method='sinc_interpolation')

down_sampled = downsample_resample(waveform)

print(down_sampled)

torchaudio.save(
    'C:/Users/hennm/Dropbox/Studium/Master/Semester 2/Tec Lab/Music/Samples/temp.wav', torch.clamp(down_sampled, -1, 1), downsample_rate, precision=32)

waveform2, sample_rate2 = torchaudio.load('temp.wav')

print(waveform2)

print((down_sampled - waveform2).abs().max())
print((down_sampled - waveform2).abs().median())

In [ ]:
'''

model = Sequential()
#model.add(LSTM(units= 1, batch_input_shape= [batch_size, timestep, 1], stateful=True, return_sequences=True))
model.add(LSTM(units= 1, activation='relu', stateful=True, return_sequences=True, batch_input_shape=(batch_size, timestep, 1)))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.fit(input_data, output_data, epochs=5, batch_size=sequence_length[0], verbose=2)
'''

In [ ]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('weights-improvement-59-2.5657-bigger.hdf5')


In [ ]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
import torchaudio
import torch
import numpy


torchaudio.set_audio_backend('sox')  # sox is the default

waveform, sample_rate = torchaudio.load('C:/Users/hennm/Dropbox/Studium/Master/Semester 2/Tec Lab/Music/Samples/Tea K Pea - nauticals_1.wav')

downsample_rate=8000

downsample_resample = torchaudio.transforms.Resample(
    sample_rate, downsample_rate, resampling_method='sinc_interpolation')

down_sampled = downsample_resample(waveform)

print(down_sampled)

torchaudio.save(
    'C:/Users/hennm/Dropbox/Studium/Master/Semester 2/Tec Lab/Music/Samples/temp.wav', torch.clamp(down_sampled, -1, 1), downsample_rate, precision=32)

waveform2, sample_rate2 = torchaudio.load('temp.wav')

print(waveform2)

print((down_sampled - waveform2).abs().max())
print((down_sampled - waveform2).abs().median())

In [ ]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
import torchaudio
import torch
import numpy


torchaudio.set_audio_backend('sox')  # sox is the default

waveform, sample_rate = torchaudio.load('C:/Users/hennm/Dropbox/Studium/Master/Semester 2/Tec Lab/Music/Samples/Tea K Pea - nauticals_1.wav')

downsample_rate=8000

downsample_resample = torchaudio.transforms.Resample(
    sample_rate, downsample_rate, resampling_method='sinc_interpolation')

down_sampled = downsample_resample(waveform)

print(down_sampled)

torchaudio.save(
    'C:/Users/hennm/Dropbox/Studium/Master/Semester 2/Tec Lab/Music/Samples/temp.wav', torch.clamp(down_sampled, -1, 1), downsample_rate, precision=32)

waveform2, sample_rate2 = torchaudio.load('temp.wav')

print(waveform2)

print((down_sampled - waveform2).abs().max())
print((down_sampled - waveform2).abs().median())

100

1

(1670, 1521)

NameError: name 'pitchnames' is not defined